In [12]:
import pandas as pd
import wandb
import os 
import sys 
__file__ = 'wandb_test.ipynd'
os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
api = wandb.Api()
entity, project = "tpn", "independent_ppo"
runs = api.runs(entity + "/" + project)
RESULTS_DIR = os.path.join(os.path.dirname(__file__), '..', 'results')
ALGORITHM_TYPES = ["independent", "centralized", "shared"]
REWARD_TYPES = ["type_1", "type_2", "type_3"]
STEP_PENALTY = [0, 0.01, 0.03]
ALGORITHM_TYPE = "independent"
REWARD_TYPE = "type_1"

This cell prints the performances of the agents in the file > results/stats_PROJECT_NAME_RUNS.txt

In [11]:
_run_stats = dict()
collated_performances_df = pd.DataFrame(
    [],
    columns = ['step_penalty', 'algorithm_type', 'reward_type', 'lstm',
        'episode_length', 'episode_reward', 'epsiode_assists',\
               'eval_length', 'eval_reward', 'eval_assists']
).fillna(0)


for run in runs:
    # if run._state != "finished":
    if run._state != "finished":
        if run._state == "running":
            _run_stats['running_runs'] = _run_stats.get('running_runs', 0) + 1
        else:
            _run_stats['crashed_runs'] = _run_stats.get('crashed_runs', 0) + 1
        continue
    _run_stats['finished_runs'] = _run_stats.get('finished_runs', 0) + 1

    config = run.config
    algo_type = config['algorithm_type']
    reward_type = config['env_config']['reward_type']
    step_penalty = config['env_config']['step_penalty']
    use_lstm = config['training']['model']['use_lstm']
    collated_performances_df.loc[len(collated_performances_df)] = [
        step_penalty,
        algo_type,
        reward_type,
        use_lstm,
        run.summary['episode_len_mean'],
        run.summary['episode_reward_mean'],
        run.summary['episode_assists_mean'],
        run.summary['eval_episode_len'],
        run.summary['eval_reward'],
        run.summary['eval_assists']
    ]

print(f"F: {_run_stats.get('finished_runs', 0)}, R: {_run_stats.get('running_runs', 0)}, C: {_run_stats.get('crashed_runs', 0)}")
collated_performances_df = collated_performances_df.groupby(['step_penalty', 'reward_type', 'lstm', 'algorithm_type']).agg(['mean', 'std', 'count'])
with open(f"{RESULTS_DIR}/stats_{project}_{_run_stats.get('finished_runs')}.txt", 'a') as f:
    f.write(f"Finished: {_run_stats.get('finished_runs', 0)}, Running: {_run_stats.get('running_runs', 0)}, Crashed: {_run_stats.get('crashed_runs', 0)}\n")
    f.write(collated_performances_df.to_string())
    f.write("\n\n\n")
    # f.write(collated_performances_df.to_latex())
    # f.write("\n\n\n")
    # f.write(collated_performances_df.to_markdown())
    # f.write("\n\n\n")
collated_performances_df
    

F: 45, R: 32, C: 0


episode_length                   \
                                                       mean        std count   
step_penalty reward_type lstm algorithm_type                                   
-0.03        type_1      True centralized         57.651250   7.118556     8   
             type_2      True centralized         60.214286  16.324253     7   
-0.01        type_1      True centralized         64.314444  12.203317     9   
             type_2      True centralized         66.054286  11.317703     7   
 0.00        type_1      True centralized         66.325714  11.450795     7   
             type_2      True centralized         69.617143   6.837328     7   

                                             episode_reward                  \
                                                       mean       std count   
step_penalty reward_type lstm algorithm_type                                  
-0.03        type_1      True centralized          2.513638  0.557952     8   
             type_2      True centralized          2.493786  1.401970     7   
-0.01        type_1      True centralized          4.354000  0.718144     9   
             type_2      True centralized          4.533271  0.576166     7   
 0.00        type_1      True centralized          5.617143  0.262851     7   
             type_2      True centralized          5.795143  0.279144     7   

                                             epsiode_assists                  \
                                                        mean       std count   
step_penalty reward_type lstm algorithm_type                                   
-0.03        type_1      True centralized           0.447500  0.224865     8   
             type_2      True centralized           0.331429  0.126416     7   
-0.01        type_1      True centralized           0.404444  0.173141     9   
             type_2      True centralized           0.318571  0.124154     7   
 0.00        type_1      True centralized           0.375714  0.150870     7   
             type_2      True centralized           0.380000  0.279404     7   

                                             eval_length                   \
                                                    mean        std count   
step_penalty reward_type lstm algorithm_type                                
-0.03        type_1      True centralized      57.256380   8.298103     8   
             type_2      True centralized      56.595288  12.376447     7   
-0.01        type_1      True centralized      65.531863   9.939226     9   
             type_2      True centralized      66.796837  11.197569     7   
 0.00        type_1      True centralized      64.675127  11.206111     7   
             type_2      True centralized      72.731404   9.529706     7   

                                             eval_reward                  \
                                                    mean       std count   
step_penalty reward_type lstm algorithm_type                               
-0.03        type_1      True centralized       2.576350  0.611049     8   
             type_2      True centralized       2.775356  1.070132     7   
-0.01        type_1      True centralized       4.375004  0.417825     9   
             type_2      True centralized       4.389908  0.600602     7   
 0.00        type_1      True centralized       5.641546  0.279243     7   
             type_2      True centralized       5.815939  0.398291     7   

                                             eval_assists                  
                                                     mean       std count  
step_penalty reward_type lstm algorithm_type                               
-0.03        type_1      True centralized        0.368001  0.171385     8  
             type_2      True centralized        0.294070  0.123283     7  
-0.01        type_1      True centralized        0.339843  0.207118     9  
             type_2      True centralized        0.19717

In [13]:
_run_stats = dict()
collated_performances_df = pd.DataFrame(
    [],
    columns = ['step_penalty', 'algorithm_type', 'reward_type', 'lstm',
        'episode_length', 'episode_reward', 'epsiode_assists',\
               'eval_length', 'eval_reward', 'eval_assists']
).fillna(0)


for run in runs:
    # if run._state != "finished":
    if run._state != "finished":
        if run._state == "running":
            _run_stats['running_runs'] = _run_stats.get('running_runs', 0) + 1
        else:
            _run_stats['crashed_runs'] = _run_stats.get('crashed_runs', 0) + 1
        continue
    _run_stats['finished_runs'] = _run_stats.get('finished_runs', 0) + 1

    config = run.config
    algo_type = config['algorithm_type']
    reward_type = config['env_config']['reward_type']
    step_penalty = config['env_config']['step_penalty']
    use_lstm = config['training']['model']['use_lstm']
    collated_performances_df.loc[len(collated_performances_df)] = [
        step_penalty,
        algo_type,
        reward_type,
        use_lstm,
        run.summary['episode_len_mean'],
        run.summary['episode_reward_mean'],
        run.summary['episode_assists_mean'],
        run.summary['eval_episode_len'],
        run.summary['eval_reward'],
        run.summary['eval_assists']
    ]

print(f"F: {_run_stats.get('finished_runs', 0)}, R: {_run_stats.get('running_runs', 0)}, C: {_run_stats.get('crashed_runs', 0)}")
collated_performances_df = collated_performances_df.groupby(['step_penalty', 'reward_type', 'lstm', 'algorithm_type']).agg(['mean', 'std', 'count'])
with open(f"{RESULTS_DIR}/stats_{project}_{_run_stats.get('finished_runs')}.txt", 'a') as f:
    f.write(f"Finished: {_run_stats.get('finished_runs', 0)}, Running: {_run_stats.get('running_runs', 0)}, Crashed: {_run_stats.get('crashed_runs', 0)}\n")
    f.write(collated_performances_df.to_string())
    f.write("\n\n\n")
    # f.write(collated_performances_df.to_latex())
    # f.write("\n\n\n")
    # f.write(collated_performances_df.to_markdown())
    # f.write("\n\n\n")
collated_performances_df
    

F: 151, R: 64, C: 0


episode_length                   \
                                                       mean        std count   
step_penalty reward_type lstm algorithm_type                                   
-0.03        type_1      True independent         51.635417   6.967563    24   
             type_2      True independent         53.847917  10.717214    24   
-0.01        type_1      True independent         53.193462   7.365092    26   
             type_2      True independent         55.211600   8.500988    25   
 0.00        type_1      True independent         54.731154   6.288050    26   
             type_2      True independent         56.288846   4.824944    26   

                                             episode_reward                  \
                                                       mean       std count   
step_penalty reward_type lstm algorithm_type                                  
-0.03        type_1      True independent          3.005913  0.518307    24   
             type_2      True independent          3.038375  1.072344    24   
-0.01        type_1      True independent          4.888527  0.350057    26   
             type_2      True independent          5.176216  0.319080    25   
 0.00        type_1      True independent          5.894615  0.090299    26   
             type_2      True independent          6.192308  0.156691    26   

                                             epsiode_assists                  \
                                                        mean       std count   
step_penalty reward_type lstm algorithm_type                                   
-0.03        type_1      True independent           0.357500  0.097546    24   
             type_2      True independent           0.406250  0.128700    24   
-0.01        type_1      True independent           0.395769  0.124873    26   
             type_2      True independent           0.534400  0.256630    25   
 0.00        type_1      True independent           0.382692  0.086836    26   
             type_2      True independent           0.517308  0.227254    26   

                                             eval_length                   \
                                                    mean        std count   
step_penalty reward_type lstm algorithm_type                                
-0.03        type_1      True independent      51.162076   7.060925    24   
             type_2      True independent      53.980393  10.778250    24   
-0.01        type_1      True independent      53.720406   8.015819    26   
             type_2      True independent      55.103922  11.214692    25   
 0.00        type_1      True independent      54.521004   7.936578    26   
             type_2      True independent      57.363776   7.757142    26   

                                             eval_reward                  \
                                                    mean       std count   
step_penalty reward_type lstm algorithm_type                               
-0.03        type_1      True independent       3.058973  0.481927    24   
             type_2      True independent       3.034287  0.956686    24   
-0.01        type_1      True independent       4.863193  0.390838    26   
             type_2      True independent       5.141932  0.401169    25   
 0.00        type_1      True independent       5.891273  0.139459    26   
             type_2      True independent       6.205917  0.178318    26   

                                             eval_assists                  
                                                     mean       std count  
step_penalty reward_type lstm algorithm_type                               
-0.03        type_1      True independent        0.313662  0.145535    24  
             type_2      True independent        0.384358  0.193158    24  
-0.01        type_1      True independent        0.417406  0.197775    26  
             type_2      True independent        0.47379

This cell prints the collated analysis df for each group [step_penalty, algo_type, reward_type] and writes them in files > [r]-[algo_type]_[reward_type]_[step_penalty]

In [24]:
import re


def get_analysis_df(algo_type, reward_type, step_penalty, use_lstm):
    file_name_prefix = "" # _chaser, _fixed, _follower
    _run_stats = dict()
    big_analysis_df = None
    run_history_df = pd.DataFrame(
        [],
            columns = [
            'run_name',
            'trial_count',
            'training_iteration',
            'episodes_total',
            'train_len',
            'train_reward',
            'train_assists',
            'eval_len',
            'eval_reward',
            'eval_assists',
            'analysis_len',
            'analysis_reward',
            'analysis_assists',]
    ).fillna(0) 
    eval_df = None
    for run in runs:
        if run._state != "finished":
            if run._state == "running":
                _run_stats['running_runs'] = _run_stats.get('running_runs', 0) + 1
            else:
                _run_stats['crashed_runs'] = _run_stats.get('crashed_runs', 0) + 1
            continue
        _run_stats['finished_runs'] = _run_stats.get('finished_runs', 0) + 1

        config = run.config
        if config['algorithm_type'] == algo_type and \
            config['env_config']['reward_type'] == reward_type and \
                config['env_config']['step_penalty'] == step_penalty and \
                    config['training']['model']['use_lstm'] == use_lstm:
            _run_stats['matching_runs'] = _run_stats.get('matching_runs', 0) + 1

            if file_name_prefix == "":
                file_name_prefix = algo_type 

            for file in run.files():
                if f"{file_name_prefix}_analysis" in file.name:
                    analysis_df = pd.read_json(
                        file.download(exist_ok=True), 
                        orient='split')
                    for file in run.files():
                        if f"{file_name_prefix}_eval" in file.name:
                            eval_stats = pd.read_json(
                                file.download(exist_ok=True), 
                                orient='split')
                            break
                    col_list = analysis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).count().columns
                    agg_funcs = []
                    for col in col_list:
                        if col == 'run_id':
                            agg_funcs.append((col, 'nunique'))
                        else:
                            agg_funcs.append((col, 'mean'))
                    analysis_df = analysis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).agg(
                        dict(agg_funcs)
                    ).reset_index()
                    analysis_df['trial_count'] = analysis_df['run_id']
                    analysis_df.drop(columns=['run_id'], inplace=True)

                    big_analysis_df = pd.concat([big_analysis_df, analysis_df]) if big_analysis_df is not None else analysis_df

                    run_history_df.loc[len(run_history_df)] = [
                        run.name,
                        analysis_df['trial_count'][0],
                        run.summary['training_iteration'],
                        run.summary['episodes_total'],
                        run.summary['episode_len_mean'],
                        run.summary['episode_reward_mean'],
                        run.summary['episode_assists_mean'],
                        run.summary['eval_episode_len'],
                        run.summary['eval_reward'],
                        run.summary['eval_assists'],
                        eval_stats['episode_len'].mean(),
                        eval_stats['episode_reward'].mean(),
                        eval_stats['assists'].mean()]
    if len(run_history_df) == 0:
        return None, None, _run_stats
    
    col_list = big_analysis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).count().columns
    agg_funcs = []
    for col in col_list:
        if col == 'run_name':
            agg_funcs.append((col, 'nunique'))
        elif col == 'trial_count':
            agg_funcs.append((col, 'sum'))
            agg_funcs.append((col, 'nunique'))
        else:
            agg_funcs.append((col, 'mean'))
    big_analysis_df = big_analysis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).agg(
        dict(agg_funcs)
    )
    run_history_df.set_index('run_name', inplace=True)  
    return big_analysis_df, run_history_df, _run_stats
                
for algo_type in ['independent', 'centralized', 'shared']:
    for reward_type in ['type_1', 'type_2', 'type_3']:
        for step_penalty in [0, 0.01, 0.03]:
            for use_lstm in [True, False]:
                analysis_df, run_history_df, _run_stats = get_analysis_df(
                algo_type,
                reward_type,
                step_penalty,
                use_lstm)
                print(f"{algo_type}, {reward_type}, {step_penalty}: {_run_stats}")

                if analysis_df is None:
                    continue

                file_name = f"{algo_type}_{reward_type}_{step_penalty}"
                if use_lstm:
                    file_name  = "r-" + file_name
                file_name = project+"_"+file_name +".txt"
                with open(f"{RESULTS_DIR}/{file_name}", 'w') as f:
                    descr = f"""{algo_type}, {reward_type}, {step_penalty}: {_run_stats}
                    COMPLETED RUNS: {_run_stats.get('finished_runs', 0)}
                    RUNNING RUNS: {_run_stats.get('running_runs', 0)}
                    CRASHED RUNS: {_run_stats.get('crashed_runs', 0)}
                    TOTAL = {_run_stats.get('finished_runs', 0) + _run_stats.get('running_runs', 0) + _run_stats.get('crashed_runs', 0)}
                    \nBASE PERFORMANCE: {run_history_df.mean()}
                    """
                    print(descr)
                    f.write(descr)
                    f.write(analysis_df.to_string())
                    f.write(run_history_df.to_string())
                #     f.write("\n\n\n")
                #     f.write(run_history_df.to_latex())
                #     f.write("\n\n\n")
                #     f.write(run_history_df.to_markdown())
                #     f.write("\n\n\n")
 

independent, type_1, 0: {'running_runs': 64, 'finished_runs': 151, 'matching_runs': 26}
independent, type_1, 0: {'running_runs': 64, 'finished_runs': 151, 'matching_runs': 26}
                    COMPLETED RUNS: 151
                    RUNNING RUNS: 64
                    CRASHED RUNS: 0
                    TOTAL = 215
                    
BASE PERFORMANCE: trial_count               2.000000
training_iteration     9890.384615
episodes_total        24935.923077
train_len                54.731154
train_reward              5.894615
train_assists             0.382692
eval_len                 54.521004
eval_reward               5.891273
eval_assists              0.346307
analysis_len             57.125416
analysis_reward           5.795561
analysis_assists          0.399481
dtype: float64
                    
independent, type_1, 0: {'running_runs': 64, 'finished_runs': 151}
independent, type_1, 0.01: {'running_runs': 64, 'finished_runs': 151}
independent, type_1, 0.01: {'running_runs': 64,

In [18]:
# simple solution that works; use it to verfiy results 
file_name_prefix = "" # _chaser, _fixed, _follower
_run_stats = dict()
collated_df = None
eval_df = None
for run in runs:
    # if run._state != "finished":
    if run._state != "finished":
        if run._state == "running":
            _run_stats['running_runs'] = _run_stats.get('running_runs', 0) + 1
        else:
            _run_stats['crashed_runs'] = _run_stats.get('crashed_runs', 0) + 1
        continue
    _run_stats['finished_runs'] = _run_stats.get('finished_runs', 0) + 1

    config = run.config
    if config['algorithm_type'] == ALGORITHM_TYPE and \
        config['env_config']['reward_type'] == REWARD_TYPE:
        _run_stats['matching_runs'] = _run_stats.get('matching_runs', 0) + 1

        if file_name_prefix == "":
            file_name_prefix = ALGORITHM_TYPE

        for file in run.files():
            if f"{file_name_prefix}_analysis" in file.name:
                analysis_table = pd.read_json(
                    file.download(exist_ok=True), 
                    orient='split')
                if collated_df is None:
                    collated_df = analysis_table
                else:
                    collated_df = pd.concat([collated_df, analysis_table])
            elif f"{file_name_prefix}_eval_df" in file.name:
                eval_stats = pd.read_json(
                    file.download(exist_ok=True), 
                    orient='split')
                if eval_df is None:
                    eval_df = eval_stats
                else:
                    eval_df = pd.concat([eval_df, eval_stats])
grouped_by_runs = collated_df.groupby(['mode', 'agent_a', 'test', 'dimension'])
grouped_by_runs[[str(i) for i in range(500, 5001, 500)]].agg(['mean', 'std', 'count'])

500                      1000  \
                                            mean       std count      mean   
mode        agent_a    test  dimension                                       
independent predator_0 ccm_1 PCA_1      0.434704  0.082415   152  0.426408   
                             PCA_2      0.345928  0.084017   152  0.351322   
                             dx         0.156349  0.078886   152  0.091855   
                             dy         0.147697  0.081587   152  0.089434   
            predator_1 ccm_1 PCA_1      0.422447  0.104933   152  0.423882   
                             PCA_2      0.350816  0.090495   152  0.358526   
                             dx         0.147816  0.081107   152  0.097993   
                             dy         0.155230  0.085229   152  0.091395   

                                                            1500            \
                                             std count      mean       std   
mode        agent_a    test  dimension                                       
independent predator_0 ccm_1 PCA_1      0.074079   152  0.424671  0.065261   
                             PCA_2      0.068521   152  0.353632  0.061704   
                             dx         0.066945   152  0.076888  0.052621   
                             dy         0.059895   152  0.070533  0.053862   
            predator_1 ccm_1 PCA_1      0.087602   152  0.418908  0.084436   
                             PCA_2      0.077430   152  0.360441  0.070641   
                             dx         0.056999   152  0.072105  0.052698   
                             dy         0.065887   152  0.074862  0.050905   

                                                  2000  ...  3500      4000  \
                                       count      mean  ... count      mean   
mode        agent_a    test  dimension                  ...                   
independent predator_0 ccm_1 PCA_1       152  0.426072  ...   152  0.423211   
                             PCA_2       152  0.355414  ...   152  0.357743   
                             dx          152  0.058934  ...   152  0.043217   
                             dy          152  0.055053  ...   152  0.036836   
            predator_1 ccm_1 PCA_1       152  0.418250  ...   152  0.415836   
                             PCA_2       152  0.361355  ...   152  0.361855   
                             dx          152  0.054954  ...   152  0.038072   
                             dy          152  0.056803  ...   152  0.042868   

                                                            4500            \
                                             std count      mean       std   
mode        agent_a    test  dimension                                       
independent predator_0 ccm_1 PCA_1      0.059368   152  0.422539  0.055980   
                             PCA_2      0.056007   152  0.357697  0.056432   
                             dx         0.043636   152  0.038507  0.038904   
                             dy         0.035146   152  0.038007  0.040119   
            predator_1 ccm_1 PCA_1      0.071870   152  0.412987  0.071766   
                             PCA_2      0.063211   152  0.363125  0.063429   
                             dx         0.040659   152  0.036921  0.040328   
                             dy         0.041566   152  0.034007  0.036728   

                                                  5000                  
                                       count      mean       std count  
mode        agent_a    test  dimension                                  
independent predator_0 ccm_1 PCA_1       152  0.421441  0.052822   152  
                             PCA_2       152  0.357467  0.057701   152  
                             dx          152  0.036296  0.038878   152  
                             dy          152  0.033678  0.037707   152  
            predator_1 ccm_1 PCA_1       152  0.412651  0.069692   152  
              

In [23]:
grouped_by_runs.mean()

TypeError: agg function failed [how->mean,dtype->object]

In [7]:
# results from fixed strategies; CALIBRATION
TEST_NAME = "ccm_1"
POLICY_SETS = ["chaser_chaser", "chaser_fixed", "chaser_follower", "fixed_follower", "follower_chaser"]
grouped_dfs = []
eval_dfs = None
for policy_set in POLICY_SETS:
    analyis_df = pd.read_csv(f"../experiments/results/{policy_set}_analysis.csv")
    eval_df = pd.read_csv(f"../experiments/results/{policy_set}_eval.csv")
    col_list = list(analyis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).mean().columns)
    agg_funs = []
    for col in col_list:
        if col != 'run_id':
            agg_funs.append((col, lambda x: round(x.mean(), 2)))
        else:
            agg_funs.append((col, 'count'))
        
    df2 = analyis_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).agg(dict(agg_funs))
    grouped_dfs.append(df2) 
    eval_dfs = pd.concat([
        eval_dfs,
        pd.DataFrame([dict(name=policy_set,**dict(eval_df.mean()))])
    ]) if eval_dfs is not None else pd.DataFrame([dict(name=policy_set,**dict(eval_df.mean()))])

    if policy_set in ["fixed_follower", "chaser_fixed", "chaser_follower"]:
        for dim in ["dx", "dy", "PCA_2"]:
            new_data = df2.loc[policy_set, 'predator_0', :, TEST_NAME, dim].T[1:]
            new_data.index.name = 'X'
            new_data.columns = [dim]
            dat_file = f"../results/data/{TEST_NAME}_{policy_set}_{dim}.dat"
            new_data.to_csv(dat_file, sep=' ')

eval_dfs.set_index('name', inplace=True)
with open("fixed_results.txt", "w") as f:
    f.write("EVALUATION simple map size 15, reward type 1, step penalty None\n")
    f.write(eval_dfs.to_string())
    f.write("\n\n")
    for df in grouped_dfs:
        f.write(df.to_string())
        f.write("\n\n")